In [95]:
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
data = []
try:
    with open('/Users/mac/Desktop/Genre Classification Dataset/train_data.txt', 'r', encoding = 'utf-8') as file:
        for line in file:
            parts = line.strip().split(":::")
            if len(parts) == 4:
                movie_id, title, genre, plot_summary = parts
                data.append({
                "movie_id": int(movie_id.strip()),
                "title": title.strip(),
                "genre": genre.strip(),
                "plot_summary": plot_summary.strip(),
            })
        file_contents = file.read()

except FileNotFoundError:
    print(f"The file '{file_path}' was not found")

except Exception as e:
    print(f"An error occured: {str(e)}")

In [97]:
df = pd.DataFrame(data)

In [98]:
def remove_punctuation(text):
    translator = str.maketrans(' ', ' ', string.punctuation)
    cleaned_text = text.translate(translator)
    return cleaned_text

In [99]:
nltk.download('stopwords')
def remove_stopword(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [101]:
from sklearn.model_selection import train_test_split

In [102]:
X = df['plot_summary'].apply(remove_punctuation)
X = X.apply(remove_stopword)
y = df['genre'].apply(remove_punctuation)
y = y.apply(remove_stopword)

In [103]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000) 
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 42)

In [105]:
from sklearn.linear_model import LogisticRegression

In [106]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [107]:
y_pred = model.predict(X_test)

In [108]:
from sklearn.metrics import accuracy_score, classification_report

In [109]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.58


In [110]:
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:\n', classification_rep)

/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
               precision    recall  f1-score   support

      action       0.48      0.24      0.32       263
       adult       0.81      0.22      0.35       112
   adventure       0.43      0.14      0.21       139
   animation       0.64      0.09      0.15       104
   biography       0.00      0.00      0.00        61
      comedy       0.51      0.59      0.55      1443
       crime       0.20      0.01      0.02       107
 documentary       0.67      0.84      0.74      2659
       drama       0.54      0.78      0.64      2697
      family       0.46      0.08      0.14       150
     fantasy       0.00      0.00      0.00        74
    gameshow       0.94      0.42      0.59        40
     history       0.00      0.00      0.00        45
      horror       0.63      0.54      0.58       431
       music       0.65      0.48      0.55       144
     musical       0.25      0.02      0.04        50
     mystery       0.00      0.00      0.00        56
   

/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [111]:
import joblib

joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


joblib.dump(model, 'logistic_regression_model.pkl')


['logistic_regression_model.pkl']

In [113]:
input_summary = "To help their unemployed father make ends meet, Edith and her twin sister Grace work as seamstresses . An invalid, Grace falls prey to the temptations of Chinatown opium and becomes an addict, a condition worsened by a misguided physician who prescribes morphine to ease her pain. When their father strikes oil, the family enjoys a new prosperity and the sisters meet the eligible Jack Herron, a fellow oil prospector. To Grace's shock, Jack falls in love with Edith and in her jealousy, Grace tells Jack that Edith, not she, has a drug problem. Hinting that her sister will soon need more morphine, Grace arranges for a dinner in Chinatown with the couple. While her sister and Jack dance, Grace slips away to an opium den. Edith follows her, but ends up in the wrong den and is arrested in an ensuing drug raid. After he bails her out of jail, Edith takes an angry Jack to search for Grace and stumbles across her half-conscious body lying in the street. The truth about the sisters is revealed, and after sending Grace to a sanitarium in the country, Jack and Edith are married."
translator = str.maketrans('', '', string.punctuation)
cleaned_summary = input_summary.translate(translator)
words = nltk.word_tokenize(cleaned_summary)
stop_words = set(nltk.corpus.stopwords.words('english'))
filtered_words = [word for word in words if word.lower() not in stop_words]
cleaned_text = ' '.join(filtered_words)


tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')


input_tfidf = tfidf_vectorizer.transform([cleaned_text])


logistic_regression_model = joblib.load('logistic_regression_model.pkl')


predicted_genre = logistic_regression_model.predict(input_tfidf)
print(f'Predicted Genre: {predicted_genre[0]}')


Predicted Genre: drama
